In [2]:
import warnings
warnings.filterwarnings(action='ignore')
%config Computer.use_jedi = False
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import missingno as msno
plt.rc('font', family='D2Coding')
from plotnine import *
import folium
from folium.plugins import MiniMap
from geopy.geocoders import Nominatim
from geopy.geocoders import Nominatim
import time
import requests

In [3]:
house = pd.read_csv('./data/아파트 실거래가.csv')
house

,시군구,번지,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명,해제사유발생일,등기일자,거래유형,중개사소재지
0,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성아파트1동~8동,79.97,202204,12,"220,000",4,1987,언주로 3,NaN,,중개거래,서울 강남구
1,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성아파트1동~8동,79.97,202204,21,"220,000",2,1987,언주로 3,NaN,,중개거래,서울 강남구
2,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성아파트1동~8동,79.97,202205,27,"216,000",2,1987,언주로 3,NaN,,중개거래,서울 강남구
3,서울특별시 강남구 개포동,1282,1282.0,0.0,개포래미안포레스트,102.32,202204,1,"369,000",13,2020,개포로 264,NaN,,중개거래,"서울 강남구, 서울 양천구"
4,서울특별시 강남구 개포동,1282,1282.0,0.0,개포래미안포레스트,136.06,202205,2,"420,000",17,2020,개포로 264,NaN,,중개거래,서울 강남구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12680,서울특별시 중랑구 중화동,450,450.0,0.0,한신아파트(103~109),59.76,202203,27,"73,000",20,1997,동일로 752,NaN,,중개거래,서울 중랑구
12681,서울특별시 중랑구 중화동,450,450.0,0.0,한신아파트(103~109),59.76,202207,20,"74,000",3,1997,동일로 752,NaN,,중개거래,서울 중랑구
12682,서울특별시 중랑구 중화동,450,450.0,0.0,한신아파트(103~109),84.03,202207,27,"91,500",12,1997,동일로 752,NaN,,중개거래,서울 중랑구
12683,서울특별시 중랑구 중화동,274-51,274.0,51.0,한영(101),84.69,202204,9,"49,900",7,2003,동일로144길 74,NaN,,중개거래,서울 중랑구


In [4]:
house.drop(['번지', '본번', '부번','층', '계약년월', '계약일' , '층', '건축년도', '해제사유발생일', '등기일자', '거래유형', '중개사소재지'] ,axis=1, inplace=True)
house.columns

Index(['시군구', '단지명', '전용면적(㎡)', '거래금액(만원)', '도로명'], dtype='object')

In [5]:
house

,시군구,단지명,전용면적(㎡),거래금액(만원),도로명
0,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,"220,000",언주로 3
1,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,"220,000",언주로 3
2,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,"216,000",언주로 3
3,서울특별시 강남구 개포동,개포래미안포레스트,102.32,"369,000",개포로 264
4,서울특별시 강남구 개포동,개포래미안포레스트,136.06,"420,000",개포로 264
...,...,...,...,...,...
12680,서울특별시 중랑구 중화동,한신아파트(103~109),59.76,"73,000",동일로 752
12681,서울특별시 중랑구 중화동,한신아파트(103~109),59.76,"74,000",동일로 752
12682,서울특별시 중랑구 중화동,한신아파트(103~109),84.03,"91,500",동일로 752
12683,서울특별시 중랑구 중화동,한영(101),84.69,"49,900",동일로144길 74


In [ ]:
# # 데이터프레임 생성 (house 데이터프레임 예시)
# house_df = house

# # 네이버 개발자 포털에서 발급한 Client ID와 Client Secret
# client_id = '5j924usqjd'
# client_secret = 'W4wjDI2jRzuvP1N0zo9UiG1zTRoDP30idjpESbp4'

# # '위도'와 '경도' 칼럼을 추가합니다.
# house_df['위도'] = None
# house_df['경도'] = None

# # 네이버 지오코딩 API를 호출하여 주소별 위도와 경도를 얻습니다.
# for index, row in house_df.iterrows():
#     address = row['도로명']
#     url = f'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query={address}'
#     headers = {
#         'X-NCP-APIGW-API-KEY-ID': client_id,
#         'X-NCP-APIGW-API-KEY': client_secret
#     }
#     response = requests.get(url, headers=headers)
    
#     if response.status_code == 200:
#         data = response.json()
#         if data.get('addresses'):
#             latitude = data['addresses'][0]['y']
#             longitude = data['addresses'][0]['x']
#             house_df.at[index, '위도'] = latitude
#             house_df.at[index, '경도'] = longitude

# # 결과를 출력합니다.
# house_df

In [7]:
house_df = pd.read_csv('./data/실거래가_위도경도.csv')
house_df

,시군구,단지명,전용면적(㎡),거래금액(만원),도로명,위도,경도
0,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,"220,000",언주로 3,37.476339,127.057140
1,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,"220,000",언주로 3,37.476339,127.057140
2,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,"216,000",언주로 3,37.476339,127.057140
3,서울특별시 강남구 개포동,개포래미안포레스트,102.32,"369,000",개포로 264,37.478454,127.054032
4,서울특별시 강남구 개포동,개포래미안포레스트,136.06,"420,000",개포로 264,37.478454,127.054032
...,...,...,...,...,...,...,...
12680,서울특별시 중랑구 중화동,한신아파트(103~109),59.76,"73,000",동일로 752,37.597236,127.082349
12681,서울특별시 중랑구 중화동,한신아파트(103~109),59.76,"74,000",동일로 752,37.597236,127.082349
12682,서울특별시 중랑구 중화동,한신아파트(103~109),84.03,"91,500",동일로 752,37.597236,127.082349
12683,서울특별시 중랑구 중화동,한영(101),84.69,"49,900",동일로144길 74,37.606457,127.082463


In [8]:
house_df_test = house_df[:5]
house_df_test

,시군구,단지명,전용면적(㎡),거래금액(만원),도로명,위도,경도
0,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,"220,000",언주로 3,37.476339,127.057140
1,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,"220,000",언주로 3,37.476339,127.057140
2,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,"216,000",언주로 3,37.476339,127.057140
3,서울특별시 강남구 개포동,개포래미안포레스트,102.32,"369,000",개포로 264,37.478454,127.054032
4,서울특별시 강남구 개포동,개포래미안포레스트,136.06,"420,000",개포로 264,37.478454,127.054032


In [15]:
# # 네이버 개발자 포털에서 발급한 Client ID와 Client Secret
# client_id = '5j924usqjd'
# client_secret = 'W4wjDI2jRzuvP1N0zo9UiG1zTRoDP30idjpESbp4'

# house_df_test['주소'] = None

# # '위도'와 '경도' 정보가 있는 데이터프레임 (house_df)에서 반복적으로 역 지오코딩을 수행합니다.
# for index, row in house_df_test.iterrows():
#     latitude = row['위도']
#     longitude = row['경도']
# #     https://naveropenapi.apigw.ntruss.com/map-reversegeocode/v2/gc?request=coordsToaddr&coords=129.1133567,35.2982640&sourcecrs=epsg:4326&output=json&orders=addr,admcode
#     if latitude is not None and longitude is not None:
#         url = f'https://naveropenapi.apigw.ntruss.com/map-reversegeocode/v2/gc?request=coordsToaddr&coords={longitude},{latitude}&sourcecrs=epsg:4326&output=json&orders=addr,admcode'
#         headers = {
#             'X-NCP-APIGW-API-KEY-ID': client_id,
#             'X-NCP-APIGW-API-KEY': client_secret
#         }
#         response = requests.get(url, headers=headers)
#         print(response)
#         if response.status_code == 200:
#             data = response.json()
#             if data.get('results'):
#                 address = data['results'][0]['region']['area3']['name']  # 예시로 주소 중 'area3'의 'name' 사용
#                 house_df_test.at[index, '주소'] = address  # 주소 정보를 데이터프레임에 추가
# house_df_test

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


,시군구,단지명,전용면적(㎡),거래금액(만원),도로명,위도,경도,주소
0,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,"220,000",언주로 3,37.476339,127.057140,개포동
1,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,"220,000",언주로 3,37.476339,127.057140,개포동
2,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,"216,000",언주로 3,37.476339,127.057140,개포동
3,서울특별시 강남구 개포동,개포래미안포레스트,102.32,"369,000",개포로 264,37.478454,127.054032,개포동
4,서울특별시 강남구 개포동,개포래미안포레스트,136.06,"420,000",개포로 264,37.478454,127.054032,개포동


In [42]:
house_df.drop(['위도', '경도'], axis=1, inplace=True)
house_df

,시군구,단지명,전용면적(㎡),거래금액(만원),도로명,주소
0,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,"220,000",언주로 3,None
1,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,"220,000",언주로 3,None
2,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,"216,000",언주로 3,None
3,서울특별시 강남구 개포동,개포래미안포레스트,102.32,"369,000",개포로 264,None
4,서울특별시 강남구 개포동,개포래미안포레스트,136.06,"420,000",개포로 264,None
...,...,...,...,...,...,...
12680,서울특별시 중랑구 중화동,한신아파트(103~109),59.76,"73,000",동일로 752,None
12681,서울특별시 중랑구 중화동,한신아파트(103~109),59.76,"74,000",동일로 752,None
12682,서울특별시 중랑구 중화동,한신아파트(103~109),84.03,"91,500",동일로 752,None
12683,서울특별시 중랑구 중화동,한영(101),84.69,"49,900",동일로144길 74,None


In [45]:
import pandas as pd
for index, row in house.iterrows():
    address = row['시군구']
    parts = address.split(' ')  # 공백으로 문자열 분할
    for part in parts:
        if '동' in part:  # '동'이 포함된 문자열 찾기
            house.at[index, '법정동'] = part
            break

# 결과를 출력합니다.
house


,시군구,단지명,전용면적(㎡),거래금액(만원),도로명,동,법정동
0,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,"220,000",언주로 3,개포동,개포동
1,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,"220,000",언주로 3,개포동,개포동
2,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,"216,000",언주로 3,개포동,개포동
3,서울특별시 강남구 개포동,개포래미안포레스트,102.32,"369,000",개포로 264,개포동,개포동
4,서울특별시 강남구 개포동,개포래미안포레스트,136.06,"420,000",개포로 264,개포동,개포동
...,...,...,...,...,...,...,...
12680,서울특별시 중랑구 중화동,한신아파트(103~109),59.76,"73,000",동일로 752,중화동,중화동
12681,서울특별시 중랑구 중화동,한신아파트(103~109),59.76,"74,000",동일로 752,중화동,중화동
12682,서울특별시 중랑구 중화동,한신아파트(103~109),84.03,"91,500",동일로 752,중화동,중화동
12683,서울특별시 중랑구 중화동,한영(101),84.69,"49,900",동일로144길 74,중화동,중화동


In [46]:
house.drop(['동'], axis=1, inplace=True)
house

,시군구,단지명,전용면적(㎡),거래금액(만원),도로명,법정동
0,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,"220,000",언주로 3,개포동
1,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,"220,000",언주로 3,개포동
2,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,"216,000",언주로 3,개포동
3,서울특별시 강남구 개포동,개포래미안포레스트,102.32,"369,000",개포로 264,개포동
4,서울특별시 강남구 개포동,개포래미안포레스트,136.06,"420,000",개포로 264,개포동
...,...,...,...,...,...,...
12680,서울특별시 중랑구 중화동,한신아파트(103~109),59.76,"73,000",동일로 752,중화동
12681,서울특별시 중랑구 중화동,한신아파트(103~109),59.76,"74,000",동일로 752,중화동
12682,서울특별시 중랑구 중화동,한신아파트(103~109),84.03,"91,500",동일로 752,중화동
12683,서울특별시 중랑구 중화동,한영(101),84.69,"49,900",동일로144길 74,중화동


In [47]:
# '거래금액(만원)' 열에서 쉼표(,)를 제거하고 문자열을 숫자로 변환
house['거래금액(만원)'] = house['거래금액(만원)'].str.replace(',', '').astype(float)
house

,시군구,단지명,전용면적(㎡),거래금액(만원),도로명,법정동
0,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,220000.0,언주로 3,개포동
1,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,220000.0,언주로 3,개포동
2,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,216000.0,언주로 3,개포동
3,서울특별시 강남구 개포동,개포래미안포레스트,102.32,369000.0,개포로 264,개포동
4,서울특별시 강남구 개포동,개포래미안포레스트,136.06,420000.0,개포로 264,개포동
...,...,...,...,...,...,...
12680,서울특별시 중랑구 중화동,한신아파트(103~109),59.76,73000.0,동일로 752,중화동
12681,서울특별시 중랑구 중화동,한신아파트(103~109),59.76,74000.0,동일로 752,중화동
12682,서울특별시 중랑구 중화동,한신아파트(103~109),84.03,91500.0,동일로 752,중화동
12683,서울특별시 중랑구 중화동,한영(101),84.69,49900.0,동일로144길 74,중화동


In [48]:
# '거래금액(만원)' 열에서 쉼표(,)를 제거하고 문자열을 숫자로 변환
house['전용면적(㎡)'] = house['전용면적(㎡)'].round()
house

,시군구,단지명,전용면적(㎡),거래금액(만원),도로명,법정동
0,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,80.0,220000.0,언주로 3,개포동
1,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,80.0,220000.0,언주로 3,개포동
2,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,80.0,216000.0,언주로 3,개포동
3,서울특별시 강남구 개포동,개포래미안포레스트,102.0,369000.0,개포로 264,개포동
4,서울특별시 강남구 개포동,개포래미안포레스트,136.0,420000.0,개포로 264,개포동
...,...,...,...,...,...,...
12680,서울특별시 중랑구 중화동,한신아파트(103~109),60.0,73000.0,동일로 752,중화동
12681,서울특별시 중랑구 중화동,한신아파트(103~109),60.0,74000.0,동일로 752,중화동
12682,서울특별시 중랑구 중화동,한신아파트(103~109),84.0,91500.0,동일로 752,중화동
12683,서울특별시 중랑구 중화동,한영(101),85.0,49900.0,동일로144길 74,중화동


In [50]:
house['제곱미터당 가격(만원)'] = (house['거래금액(만원)'] / house['전용면적(㎡)']).round()
house

,시군구,단지명,전용면적(㎡),거래금액(만원),도로명,법정동,제곱미터당 가격,제곱미터당 가격(만원)
0,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,80.0,220000.0,언주로 3,개포동,2750.0,2750.0
1,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,80.0,220000.0,언주로 3,개포동,2750.0,2750.0
2,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,80.0,216000.0,언주로 3,개포동,2700.0,2700.0
3,서울특별시 강남구 개포동,개포래미안포레스트,102.0,369000.0,개포로 264,개포동,3618.0,3618.0
4,서울특별시 강남구 개포동,개포래미안포레스트,136.0,420000.0,개포로 264,개포동,3088.0,3088.0
...,...,...,...,...,...,...,...,...
12680,서울특별시 중랑구 중화동,한신아파트(103~109),60.0,73000.0,동일로 752,중화동,1217.0,1217.0
12681,서울특별시 중랑구 중화동,한신아파트(103~109),60.0,74000.0,동일로 752,중화동,1233.0,1233.0
12682,서울특별시 중랑구 중화동,한신아파트(103~109),84.0,91500.0,동일로 752,중화동,1089.0,1089.0
12683,서울특별시 중랑구 중화동,한영(101),85.0,49900.0,동일로144길 74,중화동,587.0,587.0


In [52]:
house.drop(['전용면적(㎡)', '거래금액(만원)', '도로명', '시군구', '제곱미터당 가격', '단지명'], axis=1, inplace=True)
house

KeyError: "['시군구' '제곱미터당 가격' '단지명'] not found in axis"

In [16]:
count_df = pd.read_csv('./data/count_by_dong.csv')
count_df

,행정동명,학원 개수,면적,면적 당 학원 수,1제곱미터당 학원 수
0,가락1동,9,1340.0,0.0,0.006716
1,가락2동,22,960.0,0.0,0.022917
2,가락본동,14,1130.0,0.0,0.012389
3,가리봉동,1,400.0,0.0,0.002500
4,가산동,9,2520.0,0.0,0.003571
...,...,...,...,...,...
410,효창동,6,1120.0,0.0,0.005357
411,후암동,6,450.0,0.0,0.013333
412,휘경1동,3,530.0,0.0,0.005660
413,휘경2동,9,820.0,0.0,0.010976


In [59]:
# 네이버 개발자 포털에서 발급한 Client ID와 Client Secret
client_id = '5j924usqjd'
client_secret = 'W4wjDI2jRzuvP1N0zo9UiG1zTRoDP30idjpESbp4'

count_df['법정동'] = None

# '위도'와 '경도' 정보가 있는 데이터프레임 (house_df)에서 반복적으로 역 지오코딩을 수행합니다.
for index, row in count_df.iterrows():
    latitude = row['위도']
    longitude = row['경도']
#     https://naveropenapi.apigw.ntruss.com/map-reversegeocode/v2/gc?request=coordsToaddr&coords=129.1133567,35.2982640&sourcecrs=epsg:4326&output=json&orders=addr,admcode
    if latitude is not None and longitude is not None:
        url = f'https://naveropenapi.apigw.ntruss.com/map-reversegeocode/v2/gc?request=coordsToaddr&coords={longitude},{latitude}&sourcecrs=epsg:4326&output=json&orders=addr,admcode'
        headers = {
            'X-NCP-APIGW-API-KEY-ID': client_id,
            'X-NCP-APIGW-API-KEY': client_secret
        }
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            if data.get('results'):
                address = data['results'][0]['region']['area3']['name']  # 예시로 주소 중 'area3'의 'name' 사용
                count_df.at[index, '주소'] = address  # 주소 정보를 데이터프레임에 추가
count_df

,행정동명,학원 개수,면적,면적 당 학원 수,1제곱미터당 학원 수,위도,경도,법정동,주소
0,가락1동,9,1340.0,0.0,0.006716,37.4964898,127.1098653,None,가락동
1,가락2동,22,960.0,0.0,0.022917,37.4987054,127.1266714,None,가락동
2,가락본동,14,1130.0,0.0,0.012389,37.4955568,127.1217864,None,가락동
3,가리봉동,1,400.0,0.0,0.002500,37.4825557,126.8897489,None,가리봉동
4,가산동,9,2520.0,0.0,0.003571,37.4769094,126.8917326,None,가산동
...,...,...,...,...,...,...,...,...,...
410,효창동,6,1120.0,0.0,0.005357,37.5424569,126.9618498,None,효창동
411,후암동,6,450.0,0.0,0.013333,37.5486375,126.9781344,None,후암동
412,휘경1동,3,530.0,0.0,0.005660,37.592932,127.065725,None,휘경동
413,휘경2동,9,820.0,0.0,0.010976,37.5902093,127.0685788,None,휘경동


In [56]:
# '법정동' 열을 중심으로 그룹화하고 '학원 개수'와 '면적'을 합산합니다.
grouped_df = count_df.groupby('법정동').agg({'학원 개수': 'sum', '면적': 'sum'}).reset_index()

# 결과를 출력합니다.
grouped_df

,법정동,학원 개수,면적
0,가락동,45,3430.0
1,가리봉동,1,400.0
2,가산동,9,2520.0
3,가양동,19,6200.0
4,가회동,5,540.0
...,...,...,...
206,회기동,2,810.0
207,효창동,6,1120.0
208,후암동,6,450.0
209,휘경동,12,1350.0


In [57]:
grouped_df = grouped_df.sort_values(by='학원 개수', ascending=False)
grouped_df

,법정동,학원 개수,면적
44,대치동,701,3640.0
62,목동,327,4550.0
126,신정동,230,4290.0
176,중계동,194,9190.0
69,반포동,147,4940.0
...,...,...,...
89,삼청동,1,910.0
205,황학동,1,1050.0
157,응봉동,1,1170.0
1,가리봉동,1,400.0


In [58]:
grouped_df['1제곱킬로미터당 학원 수'] = (grouped_df['학원 개수'] / grouped_df['면적'] * 1000).round()
grouped_df

,법정동,학원 개수,면적,1제곱킬로미터당 학원 수
44,대치동,701,3640.0,193.0
62,목동,327,4550.0,72.0
126,신정동,230,4290.0,54.0
176,중계동,194,9190.0,21.0
69,반포동,147,4940.0,30.0
...,...,...,...,...
89,삼청동,1,910.0,1.0
205,황학동,1,1050.0,1.0
157,응봉동,1,1170.0,1.0
1,가리봉동,1,400.0,2.0


In [41]:
grouped_df.drop(['1제곱미터당 학원 수'], axis=1, inplace=True)
grouped_df

,법정동,학원 개수,면적,1제곱킬로미터당 학원 수
44,대치동,701,3640.0,193.0
62,목동,327,4550.0,72.0
126,신정동,230,4290.0,54.0
176,중계동,194,9190.0,21.0
69,반포동,147,4940.0,30.0
...,...,...,...,...
89,삼청동,1,910.0,1.0
205,황학동,1,1050.0,1.0
157,응봉동,1,1170.0,1.0
1,가리봉동,1,400.0,2.0


In [ ]:
# 네이버 개발자 포털에서 발급한 Client ID와 Client Secret
client_id = '5j924usqjd'
client_secret = 'W4wjDI2jRzuvP1N0zo9UiG1zTRoDP30idjpESbp4'

house_df_test['주소'] = None

# '위도'와 '경도' 정보가 있는 데이터프레임 (house_df)에서 반복적으로 역 지오코딩을 수행합니다.
for index, row in house_df_test.iterrows():
    latitude = row['위도']
    longitude = row['경도']
#     https://naveropenapi.apigw.ntruss.com/map-reversegeocode/v2/gc?request=coordsToaddr&coords=129.1133567,35.2982640&sourcecrs=epsg:4326&output=json&orders=addr,admcode
    if latitude is not None and longitude is not None:
        url = f'https://naveropenapi.apigw.ntruss.com/map-reversegeocode/v2/gc?request=coordsToaddr&coords={longitude},{latitude}&sourcecrs=epsg:4326&output=json&orders=addr,admcode'
        headers = {
            'X-NCP-APIGW-API-KEY-ID': client_id,
            'X-NCP-APIGW-API-KEY': client_secret
        }
        response = requests.get(url, headers=headers)
        print(response)
        if response.status_code == 200:
            data = response.json()
            if data.get('results'):
                address = data['results'][0]['region']['area3']['name']  # 예시로 주소 중 'area3'의 'name' 사용
                house_df_test.at[index, '주소'] = address  # 주소 정보를 데이터프레임에 추가
house_df_test

In [ ]:
data = shop_seoul_cafe_concat
cafe_map = folium.Map(location=[data.위도.mean(), data.경도.mean()], zoom_start=12)

for index, cafe in data.iterrows():
    popup = folium.Popup('{}: {:,.0f}'.format(index, cafe.매장수), max_width=300)
    folium.CircleMarker(location=[cafe.위도, cafe.경도], popup=popup, radius=cafe.매장수 / 40, 
                        color='red',fill_color='blue').add_to(cafe_map)

cafe_map.save('./output/cafe_map.html')
cafe_map